In [37]:
import pandas as pd
import numpy as np
import re

In [38]:
def modifications(input_string):
    poslist = []
    modlist = []
    if "(ac)" in input_string:
        poslist.append(0)
        modlist.append('Acetyl')
        input_string_noac = input_string.replace('(ac)', '')
    else:
        input_string_noac = input_string

    while re.search("\(ox\)", input_string_noac):
        match = re.search("\(ox\)", input_string_noac)
        pos = match.span()[0]
        poslist.append(pos)
        modlist.append('Oxidation')
        input_string_noac = re.sub('\(ox\)', '', input_string_noac, count=1)

    C_positions = [pos for pos, char in enumerate(input_string_noac) if char == 'C']
    for i in C_positions:
        modlist.append("Carbamidomethyl")
        poslist.append(i+1)

    sortedposlist = sorted(poslist)
    sortedmodlist = [x for _, x in sorted(zip(poslist, modlist))]

    formattedlist = []
    for (pos, mod) in zip(sortedposlist, sortedmodlist):
        formattedlist.append(str(pos)+'|'+str(mod))

    output = '|'.join(formattedlist)
    return output

In [39]:
data = pd.read_csv('/home/robbe/multioutput_prediction/aligned_with_multimodals_conformers_unique_grouped.csv', sep=";", converters= {"CCS":pd.eval})

In [40]:
# Strip "_" from sequence
data["sequence"] = data["Modified sequence"].str.strip("_")

# Strip everything between "()" and "[]" from sequence
data["sequence"] = data["sequence"].str.replace(r"[\(\[].*?[\)\]]", "", regex=True)


In [41]:
data['Modified sequence'] = data['Modified sequence'].str.strip("_")
data['sequence'] = data['sequence'].str.replace(")", "")

In [42]:
# Parse modifications from Modified sequence
data['Modified sequence'] = data['Modified sequence'].str.replace('(Acetyl (Protein N-term))', '(ac)')
data['Modified sequence'] = data['Modified sequence'].str.replace('(Oxidation (M))', '(ox)')

In [43]:
data['modifications'] = data['Modified sequence'].apply(modifications)
data['modifications'] = data['modifications'].replace('', 'Not modified')

In [44]:
data['CCS'] = data['CCS'].apply(lambda x: np.array(x))

In [45]:
data.rename(columns={'sequence':'seq', "Charge":"charge"}, inplace=True)

In [46]:
data['tr'] = 0

In [47]:
data.to_pickle('/home/robbe/multioutput_prediction/data_with_multimodals.pkl')